### Select N-words with higher TF

In [ ]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *

In [ ]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

In [ ]:
all_tf = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    all_tf.append(tf(l))

In [ ]:
all_tf

In [ ]:
def best_words_format(tfs, n):
    output = "Maximize\nscore: "
    for i, v in enumerate(tfs.values()):
        output += f"+ {v} c{i}"
    
    output += "\n\nSubject To\n"
    output += "length:"
    for i, v in enumerate(tfs.values()):
        output += f" c{i} +"
    output = output[:-1]
    output += f"< {n}"

    output += "\n\nBinary\n"
    for i, v in enumerate(tfs.values()):
        output += f"c{i}\n"
    return output


In [ ]:
format = best_words_format(all_tf[0], 4)

with open(os.path.join(get_git_root(), r"myLibraries\ilp_outputs\ilp_in.ilp"), "w") as f:
    f.write(format)
    f.close()

os.system(f'glpsol --tmlim 100 --lp "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_in.ilp")}" -o "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol")}"')

with open(os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol"), "r") as f:
    lines = f.readlines()
    f.close()

sentences_lines = [line for line in lines if re.search(r"c\d", line)]

sorted_lines = sorted(sentences_lines, key=lambda line: int(line.split()[1][1:]))
result = [int(sorted_line.split()[3]) for sorted_line in sorted_lines]

In [ ]:
index = [i for i, value in enumerate(result) if value == 1]
index

In [ ]:
ref = []
for i, (k, v) in enumerate(all_tf[0].items()):
    ref.append([i, k, v])
ref.sort(key=lambda x: x[2], reverse=True)
res = [v[0] for v in ref[:4]]
res

### Select N-sentences maximizing words tfs

In [1]:
import pandas as pd
import sys
import os
import re
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.score import MARSCore
from MARScore.utils import *

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotat

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

subset = dataset.iloc[3:5, :]
subset

,text,summary
3,congenital adrenal hyperplasia ( cah ) refers ...,congenital adrenal hyperplasia is a group of a...
4,type 1 diabetes ( t1d ) results from the destr...,objective(s):pentoxifylline is an immunomodula...


In [3]:
all_tokens = []
all_tf = []
for indiv in subset["text"].to_list():
    o, l = tokenizeCorpus(indiv)
    v = vectorizeCorpus(o)
    v, l = cleanAll(v, l)
    tfs = tf(l)

    all_tokens.append(l)
    all_tf.append(tfs)

In [9]:
def best_sentences_format(tokens, tfs, n):
    ### sentence tfs
    i = 0
    sentences_tfs = {}
    for token in tokens:
        if i in sentences_tfs.keys():
            sentences_tfs[i] += tfs[token]
        else:
            sentences_tfs[i] = tfs[token]
        if token == ".":
            i += 1
    
    output = "Maximize\nscore:"
    for i, cur_tf in enumerate(sentences_tfs.values()):
        output += f"+ {cur_tf} s{i}"
    
    output += "\n\nSubject To\n"
    output += f"length:"
    for i in range(len(sentences_tfs.keys())):
        output += f" s{i} +"
    output = output[:-1]
    output += f"< {n}"

    output += "\n\nBinary"
    for i in range(len(sentences_tfs.keys())):
        output += f"\ns{i}"
    return output

In [18]:
format = best_sentences_format(all_tokens[1], all_tf[1], 4)

with open(os.path.join(get_git_root(), r"myLibraries\ilp_outputs\ilp_in.ilp"), "w") as f:
    f.write(format)
    f.close()

os.system(f'glpsol --tmlim 100 --lp "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_in.ilp")}" -o "{os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol")}"')

with open(os.path.join(get_git_root(), r"myLibraries/ilp_outputs/ilp_out.sol"), "r") as f:
    lines = f.readlines()
    f.close()

sentences_lines = [line for line in lines if re.search(r"s\d", line)]

sorted_lines = sorted(sentences_lines, key=lambda line: int(line.split()[1][1:]))
result = [int(sorted_line.split()[3]) for sorted_line in sorted_lines]

In [19]:
index = [i for i, value in enumerate(result) if value == 1]
index

[38, 68, 125, 132]

In [20]:
ref = []
### sentence tfs
i = 0
sentences_tfs = {}
for token in all_tokens[1]:
    if i in sentences_tfs.keys():
        sentences_tfs[i] += all_tf[1][token]
    else:
        sentences_tfs[i] = all_tf[1][token]
    if token == ".":
        i += 1


for i, (k, v) in enumerate(sentences_tfs.items()):
    ref.append([i, k, v])
ref.sort(key=lambda x: x[2], reverse=True)
res = [v[0] for v in ref[:4]]
res.sort()
res

[38, 68, 125, 132]